In [2]:
pip install yfinance

     |████████████████████████████████| 6.3MB 4.4MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.59-py2.py3-none-any.whl size=23442 sha256=8573c5bcfd9a0e7ccd7b0e02bc68c90ea9141d9aff660f224d150368cc2fc15a
  Stored in directory: /root/.cache/pip/wheels/f8/2a/0f/4b5a86e1d52e451757eb6bc17fd899629f0925c777741b6d04
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [21]:
# Loading libraries
import numpy as np
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras import callbacks
from sklearn.model_selection import train_test_split
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import datetime
import os
import yfinance as yf
from keras.models import model_from_json
import requests

In [19]:
def split_sequences(sequences, n_steps):
        X, y = list(), list()
        for i in range(len(sequences)):
            end_ix = i + n_steps
            if end_ix > len(sequences)-1:
                break
            seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
            X.append(seq_x)
            y.append(seq_y)
        return np.array(X), np.array(y)


In [5]:
def normalize_data(dataset):
        cols = dataset.columns.tolist()
        col_name = [0]*len(cols)
        for i in range(len(cols)):
            col_name[i] = i
        dataset.columns = col_name
        dtypes = dataset.dtypes.tolist()
        # orig_answers = dataset[attr_row_predict].values
        minmax = list()
        for column in dataset:
            dataset = dataset.astype({column: 'float32'})
        for i in range(len(cols)):
            col_values = dataset[col_name[i]]
            value_min = min(col_values)
            value_max = max(col_values)
            minmax.append([value_min, value_max])
        for column in dataset:
            values = dataset[column].values
            for i in range(len(values)):
                values[i] = (values[i] - minmax[column][0]) / (minmax[column][1] - minmax[column][0])
            dataset[column] = values
        dataset[column] = values
        return dataset,minmax


In [23]:
def data_split(X,y):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
        return X_train, X_test, y_train, y_test


In [8]:
def initialize_network(n_steps,n_features,optimizer='adam'):
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, n_steps, n_features)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(50, activation='relu'))
    model.add(Dense(2))
    model.compile(optimizer=optimizer, loss='mse')
    return model

In [34]:
def train_model_1(X_train,y_train,n_steps, n_features,epochs):
    model=initialize_network(n_steps=n_steps, n_features=n_features, optimizer='adam')
    model.compile(optimizer='adam', loss='mse', metrics = None)
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=len(X_train)//4, verbose=2,validation_split = 0.3)
    return model, history

In [26]:
def evaluation(exe_time,X_test, y_test,X_train, y_train,history,model,optimizer,loss):
    model.compile(optimizer=optimizer, loss=loss, metrics = None)
    test_loss = model.evaluate(X_test, y_test, verbose=0)
    train_loss = model.evaluate(X_train, y_train, verbose=0)
    eval_test_loss = round(100-(test_loss*100),1)
    eval_train_loss = round(100-(train_loss*100),1)
    eval_average_loss = round((eval_test_loss + eval_train_loss)/2,1)
    print("--- Training Report ---")
    print('Execution time: ',round(exe_time,2),'s')
    print('Testing Accuracy:',eval_test_loss,'%')
    print('Training Accuracy:',eval_train_loss,'%')
    print('Average Network Accuracy:',eval_average_loss,'%')
    return model,eval_test_loss

In [27]:
def market_predict(model,minmax,seq_len,n_features,n_steps,data,test_loss):
    pred_data = data[-1].reshape((len(data[-1]),1, n_steps, n_features))
    pred = model.predict(pred_data)[0]
    appro_loss = list()
    for i in range(len(pred)):
        pred[i] = pred[i] * (minmax[i][1] - minmax[i][0]) + minmax[i][0]
        appro_loss.append(((100-test_loss)/100) * (minmax[i][1] - minmax[i][0]))
    return pred,appro_loss


In [15]:
def create_order(pred_price,company,test_loss,appro_loss):
    open_price,close_price = pred_price[0],pred_price[1]
    if open_price > close_price:
        side = 'sell'
    elif open_price < close_price:
        side = 'buy'
    if side == 'buy':
        order = {
            'symbol':company,
            'qty':round(20*(test_loss/100)),
            'type':'stop_limit',
            'time_in_force':'day',
            'side': 'buy',
            'take_profit': close_price + appro_loss,
            'stop_loss': close_price - appro_loss
                }
    elif side == 'sell':
        order = {
            'symbol':company,
            'qty':round(20*(test_loss/100)),
            'type':'stop_limit',
            'time_in_force':'day',
            'side': 'sell',
            'take_profit':close_price - appro_loss,
            'stop_loss':close_price + appro_loss
                }
    r = requests.post(ORDERS_URL, json = order,headers = HEADERS)
    print(r.content)

In [6]:
def data_setup(symbol,data_len,seq_len):
    end = datetime.datetime.today().strftime('%Y-%m-%d')
    start = datetime.datetime.strptime(end, '%Y-%m-%d') - datetime.timedelta(days=(data_len/0.463)) # 0.463 applied as markets are closed on weekends and public holidays
    orig_dataset = yf.download(symbol,start,end)
    close = orig_dataset['Close'].values
    open_ = orig_dataset['Open'].values
    high = orig_dataset['High'].values
    low = orig_dataset['Low'].values
    dataset,minmax = normalize_data(orig_dataset)
    cols = dataset.columns.tolist()
    data_seq = list()
    for i in range(len(cols)):
        if cols[i] < 4:
            data_seq.append(dataset[cols[i]].values)
            data_seq[i] = data_seq[i].reshape((len(data_seq[i]), 1))
    data = hstack(data_seq)
    n_steps = seq_len
    X, y = split_sequences(data, n_steps)
    n_features = X.shape[2]
    n_seq = len(X)
    n_steps = seq_len
    print(X.shape)
    X = X.reshape((n_seq,1, n_steps, n_features))
    true_y = []
    for i in range(len(y)):
        true_y.append([y[i][0],y[i][1]])
    return X,array(true_y),n_features,minmax,n_steps,close,open_,high,low


In [47]:
# Basic exploration of the stock
msft = yf.Ticker("MSFT")

print("OVERALL INFO: \n", msft.info)
print("\n")

print("DIVIDENDS: \n", msft.dividends)
print("\n")

print("MAJOR SHARE HOLDERS: \n" ,msft.major_holders)
print("\n")

print("CASHFLOW: \n", msft.cashflow)
print("\n")

print("EARNINGS: \n", msft.earnings )
print("\n")

OVERALL INFO: 
 {'zip': '98052-6399', 'sector': 'Technology', 'fullTimeEmployees': 163000, 'longBusinessSummary': 'Microsoft Corporation develops, licenses, and supports software, services, devices, and solutions worldwide. Its Productivity and Business Processes segment offers Office, Exchange, SharePoint, Microsoft Teams, Office 365 Security and Compliance, and Skype for Business, as well as related Client Access Licenses (CAL); Skype, Outlook.com, OneDrive, and LinkedIn; and Dynamics 365, a set of cloud-based and on-premises business solutions for small and medium businesses, large organizations, and divisions of enterprises. Its Intelligent Cloud segment licenses SQL and Windows Servers, Visual Studio, System Center, and related CALs; GitHub that provides a collaboration platform and code hosting service for developers; and Azure, a cloud platform. It also offers support services and Microsoft consulting services to assist customers in developing, deploying, and managing Microsoft 

In [40]:
X,y,n_features,minmax,n_steps,close,open,high,low= data_setup('MSFT',365,10)

[*********************100%***********************]  1 of 1 completed
(534, 10, 4)


In [41]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.33)

In [42]:
model,history=train_model_1(X_train,y_train,n_steps, n_features,epochs=20)

Epoch 1/20
3/3 - 2s - loss: 0.2340 - val_loss: 0.1981
Epoch 2/20
3/3 - 0s - loss: 0.1703 - val_loss: 0.1358
Epoch 3/20
3/3 - 0s - loss: 0.1112 - val_loss: 0.0781
Epoch 4/20
3/3 - 0s - loss: 0.0580 - val_loss: 0.0304
Epoch 5/20
3/3 - 0s - loss: 0.0192 - val_loss: 0.0083
Epoch 6/20
3/3 - 0s - loss: 0.0096 - val_loss: 0.0139
Epoch 7/20
3/3 - 0s - loss: 0.0184 - val_loss: 0.0154
Epoch 8/20
3/3 - 0s - loss: 0.0158 - val_loss: 0.0077
Epoch 9/20
3/3 - 0s - loss: 0.0073 - val_loss: 0.0043
Epoch 10/20
3/3 - 0s - loss: 0.0047 - val_loss: 0.0058
Epoch 11/20
3/3 - 0s - loss: 0.0056 - val_loss: 0.0071
Epoch 12/20
3/3 - 0s - loss: 0.0059 - val_loss: 0.0062
Epoch 13/20
3/3 - 0s - loss: 0.0046 - val_loss: 0.0041
Epoch 14/20
3/3 - 0s - loss: 0.0030 - val_loss: 0.0028
Epoch 15/20
3/3 - 0s - loss: 0.0025 - val_loss: 0.0027
Epoch 16/20
3/3 - 0s - loss: 0.0031 - val_loss: 0.0027
Epoch 17/20
3/3 - 0s - loss: 0.0030 - val_loss: 0.0022
Epoch 18/20
3/3 - 0s - loss: 0.0023 - val_loss: 0.0018
Epoch 19/20
3/3 - 0

In [43]:
_,eval_test_loss=evaluation(exe_time=2,X_test=X_test, y_test=y_test,
                                X_train=X_train, y_train= y_train,
                                history=history,model= model,optimizer='adam',loss='mse')

--- Training Report ---
Execution time:  2 s
Testing Accuracy: 99.8 %
Training Accuracy: 99.8 %
Average Network Accuracy: 99.8 %
